In [104]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import folium
from sklearn.cluster import KMeans

# ============= PART 1 ===================================

### Retrieving data  
* Building a BeautifulSoup object based on HTML response from url

In [3]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')

### Building dataframe on postal codes of Canada

In [4]:
table_contents = []
table=soup.find('table')
for row in table.find_all('tr')[1:]:
    cell = {}
    table_contents.append(row.text.strip().split('\n\n'))
toronto_data = pd.DataFrame(table_contents, columns=['PostalCode','Borough','Neighbourhood'])

In [5]:
toronto_data.shape

(180, 3)

### Removing Borough == Not assigned

In [9]:
toronto_data = toronto_data[toronto_data['Borough']!='Not assigned']

In [10]:
toronto_data.shape

(103, 3)

### Ensuring there is no cell that has a borough but a Not assigned  neighborhood

In [17]:
toronto_data[toronto_data['Neighbourhood']=='Not assigned'].shape

(0, 3)

In [18]:
toronto_data.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Final dataset shape

In [16]:
toronto_data.shape

(103, 3)

# ============= PART 2 ===================================

In [33]:
import geocoder # import geocoder

def get_coordinates(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude, longitude

In [34]:
get_coordinates('M8X')

KeyboardInterrupt: 

In [36]:
geo_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

In [37]:
geo_df.shape

(103, 3)

In [38]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [41]:
toronto_geo_data = toronto_data.merge(geo_df, left_on='PostalCode', right_on='Postal Code', how='inner')

In [43]:
toronto_geo_data.shape

(103, 6)

In [44]:
toronto_geo_data.head()

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A,43.662301,-79.389494


# ============= PART 3 ===================================

#### Create a map of Toronto with neighborhoods superimposed on top.


In [66]:
# create map of Toronto using random latitude and longitude values from Toronto
import folium
latitude = toronto_geo_data.loc[0,'Latitude']
longitude = toronto_geo_data.loc[0,'Longitude']
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_geo_data['Latitude'], toronto_geo_data['Longitude'], toronto_geo_data['Borough'], toronto_geo_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [69]:
import getpass
CLIENT_SECRET = 'VXQHIW1GN2LHTI4TECUUDCFMIVLHHRT2JVBDTMYVE1EFZ2PC'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
CLIENT_ID = getpass.getpass('Enter your CLIENT_ID')

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

Enter your CLIENT_ID········


## Explore Neighborhoods in Toronto single borough
#### Choosing the borough to do analysis

In [72]:
toronto_geo_data.groupby('Borough').count()

,PostalCode,Neighbourhood,Postal Code,Latitude,Longitude
Borough,,,,,
Central Toronto,9,9,9,9,9
Downtown Toronto,19,19,19,19,19
East Toronto,5,5,5,5,5
East York,5,5,5,5,5
Etobicoke,12,12,12,12,12
Mississauga,1,1,1,1,1
North York,24,24,24,24,24
Scarborough,17,17,17,17,17
Toronto/York,1,1,1,1,1


In [73]:
nort_york_data = toronto_geo_data[toronto_geo_data['Borough']=='North York']

In [74]:
nort_york_data.shape

(24, 6)

#### Get venues for Toronto Borough = North York


In [75]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [79]:
# type your answer here
nort_york_venues = getNearbyVenues(nort_york_data['Neighbourhood'],
                                   nort_york_data['Latitude'],
                                   nort_york_data['Longitude'])

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [81]:
print(nort_york_venues.shape)
nort_york_venues.head()

(243, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


#### check how many venues were returned for each neighborhood

In [87]:
nort_york_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Don Mills,24,24,24,24,24,24
Downsview,14,14,14,14,14,14
"Fairview, Henry Farm, Oriole",68,68,68,68,68,68
Glencairn,5,5,5,5,5,5
Hillcrest Village,5,5,5,5,5,5
Humber Summit,4,4,4,4,4,4


In [88]:
print('There are {} uniques categories.'.format(len(nort_york_venues['Venue Category'].unique())))

There are 99 uniques categories.


## Analyze Each Neighborhood for Nort York venues


In [91]:
pd.get_dummies(nort_york_venues[['Venue Category']], prefix="", prefix_sep="")

,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
239,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
240,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
241,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [94]:
# one hot encoding
nort_york_onehot = pd.get_dummies(nort_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nort_york_onehot['Neighborhood'] = nort_york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [nort_york_onehot.columns[-1]] + list(nort_york_onehot.columns[:-1])
nort_york_onehot = nort_york_onehot[fixed_columns]

nort_york_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Group rows by neighborhood and take a mean value of the frequency of occurance of each category

In [95]:
nort_york_grouped = nort_york_onehot.groupby('Neighborhood').mean().reset_index()
nort_york_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.095238,...,0.047619,0.000000,0.047619,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.040000,0.000000,0.04,0.000000,0.040000,0.000000,0.000000,0.000000
3,Don Mills,0.000000,0.000000,0.000000,0.041667,0.000000,0.041667,0.041667,0.000000,0.000000,...,0.041667,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Downsview,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.071429,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Fairview, Henry Farm, Oriole",0.014706,0.000000,0.014706,0.000000,0.000000,0.014706,0.000000,0.029412,0.029412,...,0.000000,0.014706,0.000000,0.014706,0.00,0.014706,0.014706,0.014706,0.000000,0.029412
6,Glencairn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,...,0.000000,0.000000,0.200000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,Hillcrest Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,Humber Summit,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Humberlea, Emery",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


#### Display the top 10 venues for each neighborhood.


In [102]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = nort_york_grouped['Neighborhood']

for ind in np.arange(nort_york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nort_york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Grocery Store,Restaurant,Frozen Yogurt Shop,Gas Station,Diner,Ice Cream Shop,Deli / Bodega,Middle Eastern Restaurant
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dog Run,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Grocery Store,Juice Bar,Café,Butcher,Comfort Food Restaurant,Indian Restaurant,Pharmacy
3,Don Mills,Gym,Coffee Shop,Restaurant,Grocery Store,Sandwich Place,Discount Store,Italian Restaurant,Japanese Restaurant,Dim Sum Restaurant,Clothing Store
4,Downsview,Grocery Store,Park,Airport,Discount Store,Gym / Fitness Center,Athletics & Sports,Food Truck,Bank,Home Service,Baseball Field


## Cluster Neighborhoods


In [105]:
# set number of clusters
kclusters = 5

nort_york_grouped_clustering = nort_york_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nort_york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 4])

In [131]:
kmeans.labels_ 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 3, 2, 2, 2, 1, 0])

In [153]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge nort_york_grouped with nort_york_data to add latitude/longitude for each neighborhood

nort_york_merged = nort_york_data.reset_index(drop=True)

nort_york_merged.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)
#df.rename(columns={"A": "a", "B": "c"})
nort_york_merged = nort_york_merged.merge(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='inner')

nort_york_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656,3,Park,Food & Drink Shop,Discount Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572,2,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Intersection,Diner,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
2,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763,2,Clothing Store,Furniture / Home Store,Women's Store,Miscellaneous Shop,Boutique,Coffee Shop,Vietnamese Restaurant,Accessories Store,Supplement Shop,Chinese Restaurant
3,M3B,North York,Don Mills,M3B,43.745906,-79.352188,2,Gym,Coffee Shop,Restaurant,Grocery Store,Sandwich Place,Discount Store,Italian Restaurant,Japanese Restaurant,Dim Sum Restaurant,Clothing Store
4,M3C,North York,Don Mills,M3C,43.725900,-79.340923,2,Gym,Coffee Shop,Restaurant,Grocery Store,Sandwich Place,Discount Store,Italian Restaurant,Japanese Restaurant,Dim Sum Restaurant,Clothing Store


In [155]:
nort_york_merged['Cluster Labels']

0     3
1     2
2     2
3     2
4     2
5     2
6     2
7     2
8     2
9     2
10    2
11    2
12    2
13    2
14    2
15    0
16    2
17    2
18    2
19    4
20    2
21    1
22    2
Name: Cluster Labels, dtype: int32

#### Finally, let's visualize the resulting clusters


In [156]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nort_york_merged['Latitude'], nort_york_merged['Longitude'], nort_york_merged['Neighborhood'], nort_york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters